In [24]:
def get_word_count(filename):
  with open(filename, 'r') as f:
    # Read the entire file content
    content = f.read()

    # Split the content into words, removing whitespaces
    words = content.split()

    return len(words)

# Example usage
filename = "./raw_data/combined_unlabeled_commentary_clean.txt"  # Replace with your actual filename
word_count = get_word_count(filename)
print(f"The file '{filename}' has {word_count} words.")
# print(f"The file '{filename2}' has {word_count2} words.")

The file './raw_data/combined_unlabeled_commentary_clean.txt' has 3962 words.


In [43]:
def split_sentences(input_file, output_file):
  """
  Splits a text file into separate lines for each sentence.

  Args:
    input_file (str): Path to the input text file.
    output_file (str): Path to the output file where the split sentences will be written.
  """
  sentences = []
  current_sentence = ""
  with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for line in infile:
      # Add line to current sentence (excluding trailing newline)
      current_sentence += line.strip()

      # Check for sentence delimiters (e.g., ., ?, !) and write complete sentences
      for delimiter in [".", "?", "!"]:
        if delimiter in current_sentence:
          if (len(sentences) < 100 and len(current_sentence) > 8):
            sentences.append(current_sentence.split(delimiter)[0] + delimiter + "\n")
          current_sentence = current_sentence.split(delimiter)[1].strip()

      # Write any remaining sentence at the end of the file
      # if current_sentence:
      #   sentences.append(current_sentence)
    outfile.writelines(sentences)

# Example usage:
input_file = "./raw_data/combined_unlabeled_commentary.txt"
output_file = "./raw_data/combined_unlabeled_commentary_clean.txt"  # Output file with split sentences

split_sentences(input_file, output_file)

print(f"Text file split into sentences and saved to {output_file}")

Text file split into sentences and saved to ./raw_data/combined_unlabeled_commentary_clean.txt


In [42]:
word_count = get_word_count(output_file)
word_count

# dataset

4061

In [45]:
from transformers import BertTokenizer, BertForMaskedLM
import torch
from torch.utils.data import Dataset, DataLoader
import random

# Check if CUDA is available
device = torch.device("mps" if torch.cuda.is_available() else "cpu")

# Define a custom dataset class
class TextDataset(Dataset):
  def __init__(self, filepath):
    self.sentences = []
    with open(filepath, 'r') as f:
      for line in f:
        self.sentences.append(line.strip())

  def __len__(self):
    return len(self.sentences)

  def __getitem__(self, idx):
    sentence = self.sentences[idx]
    #print(sentence)

    # Tokenize the sentence
    tokenized_sentence = tokenizer.tokenize(sentence, padding='max_length', return_tensors='pt')
    # Apply MLM (replace some tokens with [MASK])
    masked_sentence = mask_tokens(tokenized_sentence, tokenizer)
    # Convert tokens to ids
    input_ids = tokenizer.convert_tokens_to_ids(masked_sentence)
    # Convert labels (masked tokens) to ids
    labels = tokenizer.convert_tokens_to_ids([t for t in masked_sentence if t == '[MASK]'])
    attention_mask = [1] * len(input_ids)  # All tokens are attended to
    print(input_ids)

    # Move tensors to cuda device
    # input_ids = input_ids.to(device)
    # labels = labels.to(device)
    # attention_mask = attention_mask.to(device)
    return {'input_ids': input_ids.flatten(), 'attention_mask': attention_mask.flatten(),
            'label': labels.flatten()}

# Define function to mask tokens
def mask_tokens(tokens, tokenizer):
  """
  Performs MLM by masking a certain percentage of tokens.
  """
  # Replace some tokens with [MASK]
  cand_mask_idx = [i for i, t in enumerate(tokens) if t not in [tokenizer.cls_token, tokenizer.sep_token]]
  # Adjust masking rate based on your needs (e.g., 15% for BERT pre-training)
  num_to_mask = int(0.15 * len(cand_mask_idx))
  rng = torch.rand(len(cand_mask_idx))
  masked_tokens = [t if rng[i] > 0.8 else '[MASK]' for i, t in enumerate(tokens)]
  # Replace 10% of masked tokens with random words (optional)
  for i in range(num_to_mask // 10):
    masked_tokens[torch.randint(len(cand_mask_idx), size=(1,))[0]] = random.choice(list(tokenizer.vocab.keys()))
  return masked_tokens


In [46]:
# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'  # Replace with your desired model
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

# Move model to cuda device
model.to(device)

# Define training parameters
batch_size = 8
learning_rate = 2e-5
num_epochs = 3
print("here3")
# Load your text file
dataset = TextDataset(filename)
print("here2")
# Create dataloader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
print("here1")
# Define optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


here3


TypeError: '>' not supported between instances of 'str' and 'int'

In [15]:
print("here")
# Training loop
i = 0
for epoch in range(num_epochs):
  print("test2")
  for step, batch in enumerate(dataloader):
    print("test")
    # Forward pass (move tensors to device before feeding to model)
    if i%1000 == 0:
      print(tokenizer.decode(input_ids[0]))
      # print(input_ids)
      # print(labels)
      # print(attention_mask)
      
    i+=1
    input_ids = batch['input_ids'].to(device)
    labels = batch['attention_mask'].to(device)
    attention_mask = batch['label'].to(device)
    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
    loss = criterion(outputs.logits.view(-1, tokenizer.vocab_size), labels.view(-1))

    # Backward pass and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print training progress (optional)
    print(f"Epoch: {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

Keyword arguments {'padding': 'max_length', 'return_tensors': 'pt'} not recognized.


here
test2


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



AttributeError: 'list' object has no attribute 'flatten'

In [61]:
# 


SyntaxError: invalid syntax (1342594628.py, line 1)

In [17]:
from transformers import BertTokenizer, BertForMaskedLM, DataCollatorForLanguageModeling
from datasets import load_dataset
import torch
from torch.utils.data import DataLoader

# Set device to GPU if available
device = "mps" if torch.cuda.is_available() else "cpu"

# Define tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased").to(device)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [83]:
# Define custom dataset class
class MLMDataset(Dataset):
  def __init__(self, tokenizer, filepath):
    self.tokenizer = tokenizer
    self.sentences = []
    with open(filepath, "r", encoding="utf-8") as f:
      for line in f:
        # Clean and strip whitespaces from each line
        clean_line = line.strip()
        if clean_line and len(clean_line) > 8:  # Add only non-empty lines
          self.sentences.append(clean_line)

  def __len__(self):
    return len(self.sentences)

  def __getitem__(self, idx):
    sentence = self.sentences[idx]
    # print(idx)
    # print(sentence + "\n")
    encoding = self.tokenizer(sentence, padding="max_length", truncation=True, return_tensors="pt")
    print(encoding["attention_mask"])
    return encoding


# Create dataset object"
dataset = MLMDataset(tokenizer, "./raw_data/combined_unlabeled_commentary_clean.txt")

# Create DataCollator for language modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True)

# Define training loop parameters
batch_size = 8
learning_rate = 2e-5
epochs = 3

# Create DataLoader
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define optimizer
#optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(epochs):
  print(f"Epoch: {epoch+1}")
  for batch in train_dataloader:
    print("working")
    # Move batch data to device
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)
    print(input_ids.shape)
    input_ids.squeeze(0)
    print(input_ids.shape)

    # Forward pass
    loss, logits  = model(input_ids, attention_mask=attention_mask, labels=labels)
    #loss = outputs.loss

    # Backward pass and update weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print loss
    print(f"Batch loss: {loss.item():.4f}")

print("Training complete!")

In [250]:
from transformers import BertTokenizer, BertForMaskedLM, DataCollatorForLanguageModeling
from datasets import load_dataset
import torch

# Define paths and hyperparameters
model_name = "bert-base-uncased"  # Pre-trained BERT model
text_file = "./raw_data/combined_unlabeled_commentary_clean.txt"  # Path to your text file
batch_size = 8  # Adjust this based on your GPU memory
max_len = 512  # Maximum sequence length

# Check for GPU availability
device = torch.device("mps")

# Load tokenizer and model on chosen device
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name).to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [251]:
# Define function to mask tokens
def mask_sentence(sentence):
  """Masks 15% of the tokens in a sentence."""
  # print("ok")
  # print(sentence["text"])
  # print(type(sentence["text"]))
  masked_sentence = tokenizer(sentence["text"], return_tensors="pt", truncation=True, padding=True)
  masked_sentence = masked_sentence.to(device)  # Move data to device

  input_ids = masked_sentence["input_ids"]
  attention_mask = masked_sentence["attention_mask"]
  labels = input_ids.clone()

  # Randomly mask 15% of tokens
  probability_mask = 0.15
  mask_indices = torch.bernoulli(torch.ones_like(input_ids) * probability_mask).bool()
  labels[~mask_indices] = -100  # Set padding tokens to -100

  # Replace masked tokens with [MASK]
  input_ids[mask_indices] = tokenizer.mask_token_id

  return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}


In [252]:

# Load data from text file
dataset = load_dataset("text", data_files={"train": text_file}, split="train")
train_data = dataset.map(mask_sentence, batched=True)
print(type(train_data))
train_data.set_format("torch")

# Remove 'text' column and set format to 'torch'
print(type(train_data))
print(train_data)
train_data = train_data.remove_columns(["text"])
print(type(train_data))
print(train_data)
print(type(train_data[0]))

train_loader = DataLoader(train_data, batch_size=5, shuffle=True)

# print(train_data)
# #print(train_data['text'][0])
# print(train_data['input_ids'][0])
# print(train_data['attention_mask'][0])
# print(train_data['labels'][0])
# print(len(train_data))

# train_data = train_data.remove_columns(["text"])
# #train_data = train_data.set_format(type='torch', columns=['input_ids','attention_mask','labels'])

# # # Define optimizer and training loop (replace with your training logic)
# optimizer = torch.optim.AdamW(model.parameters())

# print(type(train_data))
# print(train_data)
# #print(train_data['text'][0])
# print(train_data['input_ids'][0])
# print(train_data['attention_mask'][0])
# print(train_data['labels'][0])
 

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>
Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 100
})
<class 'datasets.arrow_dataset.Dataset'>
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 100
})
<class 'dict'>


In [253]:
num_batches = len(train_dataloader)
print(f"Number of batches: {num_batches}")

Number of batches: 13


In [254]:
for epoch in range(3):  # Adjust number of epochs
  for batch in train_loader:
    # Move data to device
    #batch.to(device)
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)
    print(type(input_ids))
    print(type(attention_mask))
    print(type(labels))
    print(len(input_ids))
    print (len(attention_mask))
    print (len(labels))
    #batch = {k: v.to(device) for k, v in batch.items()}

    # Forward pass
    outputs = model(input_ids= input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss

    # Backward pass and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print training progress (optional)
    print(f"Epoch: {epoch+1}, Loss: {loss}")

# Save the fine-tuned model (optional)
model.save_pretrained("masked_lm_model")

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
5
5
5
Epoch: 1, Loss: 8.32705020904541
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
5
5
5
Epoch: 1, Loss: 10.645683288574219
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
5
5
5
Epoch: 1, Loss: 11.859464645385742
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
5
5
5
Epoch: 1, Loss: 10.321561813354492
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
5
5
5
Epoch: 1, Loss: 9.99863338470459
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
5
5
5
Epoch: 1, Loss: 11.230396270751953
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
5
5
5
Epoch: 1, Loss: 13.147407531738281
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
5
5
5
Epoch: 1, Loss: 11.418228149414062
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
5
5
5
Epoch: 1, Loss: 12.357796669006348
<class 'torch.

In [170]:
from datasets import Dataset
dataset = Dataset.from_dict({"a": ["ok", "okk", "2"]})
print(type(dataset))
print(type(dataset["a"]))
test = dataset.map(lambda batch: {"b": batch["a"]})
print(test)
#train_data = dataset.map(mask_sentence)


# second = dataset.map(lambda batch: {"b": batch["a"]}, batched=True)
# print(type(second))

<class 'datasets.arrow_dataset.Dataset'>
<class 'list'>


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Dataset({
    features: ['a', 'b'],
    num_rows: 3
})
